In [ ]:
import textual_help as th

### Sanity Check for dependencies

In [ ]:
import torch, accelerate, transformers, sentence_transformers
print(torch.__version__, accelerate.__version__, transformers.__version__, sentence_transformers.__version__)

## Import of datasets, e.g.: KIT-ML, HumanML3D

In [ ]:
h3d_df = th.read_data_texts("../datasets/humanml3d/texts")
kit_df = th.read_data_texts("../datasets/KIT-ML/texts")

In [ ]:
h3d_df

## Checking if positional arguments are the same amount as the words in description

In [ ]:
mask = h3d_df["description"].str.strip().str.split().str.len() != \
       h3d_df["pos_tags"].str.strip().str.split().str.len()

mismatch_df = h3d_df[mask]
mismatch_df

## Analysis of Datasets

In [ ]:
th.analysis_routine(h3d_df)
th.analysis_routine(kit_df)

### Specific use of IMS (already included in full routine)

In [ ]:
data_h3d = th.intramotion_similarity_nb(h3d_df)

In [ ]:
data_kit = th.intramotion_similarity_nb(kit_df)



### Setting thresholds for similarity

In [ ]:
dist_df = data_h3d["mpnet"][0]
dist_df.to_csv("h3d_intramotion_similarity_mpnet.csv", index=False)

bad_ids_0 = set(dist_df.loc[dist_df["mean_intra_caption_sim"] < 10, "motion_id"].astype(str))
bad_ids_60 = set(dist_df.loc[dist_df["mean_intra_caption_sim"] < 0.60, "motion_id"].astype(str))
bad_ids_70 = set(dist_df.loc[dist_df["mean_intra_caption_sim"] < 0.70, "motion_id"].astype(str))
bad_ids_75 = set(dist_df.loc[dist_df["mean_intra_caption_sim"] < 0.75, "motion_id"].astype(str))
bad_ids_80 = set(dist_df.loc[dist_df["mean_intra_caption_sim"] < 0.80, "motion_id"].astype(str))
bad_ids_90 = set(dist_df.loc[dist_df["mean_intra_caption_sim"] < 0.85, "motion_id"].astype(str))

print(dist_df[dist_df["motion_id"] == "00356"])
#len(bad_ids_75)
#len(bad_ids_80)
print(len(bad_ids_0))
print("\n")
print(len(bad_ids_60))
print(len(bad_ids_70))
print(len(bad_ids_80))
print(len(bad_ids_90))

### Removing "bad ids" that are sub-threshold

In [ ]:
exit(1)
import util
#util.remove_bad_ids("../datasets/humanml3d_70", bad_ids_70)
#util.remove_bad_ids("../datasets/humanml3d_75", bad_ids_75)
util.remove_bad_ids("../datasets/humanml3d_60", bad_ids_60)
util.remove_bad_ids("../datasets/humanml3d_80", bad_ids_80)

In [ ]:
import os
from pathlib import Path

bad_ids = bad_ids_60
folder = Path("../datasets/humanml3d_60/")

for bad_id in bad_ids:
    file_path = folder / f"texts/{bad_id}.txt"
    file_path_ani = folder / f"animations/{bad_id}.mp4"
    try:
        file_path.unlink()  # deletes the file
        print(f"Deleted {file_path}")
    except FileNotFoundError:
        print(f"File {file_path} not found, skipping.")

    try:
        file_path_ani.unlink()  # deletes the file
        print(f"Deleted {file_path_ani}")
    except FileNotFoundError:
        print(f"File {file_path_ani} not found, skipping.")